# Deep Learning with Tensorflow on Amazon SageMaker and S3



## Imports

In [36]:
import sagemaker

print(sagemaker.__version__)
sess = sagemaker.Session()
# role = sagemaker.get_execution_role()
import os
from tensorflow import keras
import tensorflow as tf
import numpy as np
from sagemaker.inputs import FileSystemInput

2.32.1


In [37]:
tf.__version__

'2.4.1'

## Train on the notebook on local machine

In [6]:
from sagemaker.tensorflow import TensorFlow

# put these files in the respective training folder
training_file = "labels_test_v1_m.csv"
validation_file = "val_labels_m.csv"

tf_estimator = TensorFlow(entry_point='train_no_s3.py', 
                          role="arn:aws:iam::963659202518:role/service-role/AmazonSageMaker-ExecutionRole-20210306T191865",
                          instance_count=1, 
                          instance_type='local',
                          framework_version='2.4.1', 
                          base_job_name='pc-tf-custom-container-test-job',
                          output_path='s3://canopy-production-ml-output',
                          py_version='py37',
                          script_mode=True,
                          hyperparameters={'training_file':training_file, 
                                           'validation_file': validation_file,
#                                            'bucket':"margaux-bucket-us-east-1",
                                           'epochs': 2, 'augment': True, 'batch-size': 100,
                                           'learning-rate': 0.01, 'numclasses': 10,
                                           'wandb_key': "abfa0dec9fc06fbfa6392496f40a22a8d47e58cf",
                                           's3_chkpt_dir':"ckpt",
                                           'bands': "2 3 4 8 12",
                                           'starting_checkpoint':"ckpt/pc-tf-custom-container-test-job-RGBNIRN-2021-03-15-22-45-06-142/model_resnet_epoch_7.h5"}
                         )
                                            

Training

In [5]:
import time
t0 = time.time()
tf_estimator.fit("file:///Users/purgatorid/Documents/GitHub/Project Canopy/cb_feature_detection/model-development/data")
t1 = time.time()
total = (t1-t0) / 60
print(f"{total} min")

Creating network "sagemaker-local" with the default driver
Creating g0zhkp446s-algo-1-duwzu ... 
Attaching to g0zhkp446s-algo-1-duwzu2mdone
g0zhkp446s-algo-1-duwzu | 2021-03-24 02:40:31.880169: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
g0zhkp446s-algo-1-duwzu | 2021-03-24 02:40:31.882746: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
g0zhkp446s-algo-1-duwzu | 2021-03-24 02:40:31.928211: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
g0zhkp446s-algo-1-duwzu | 2021-03-24 02:40:34,195 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
g0zhkp446s-algo-1-duwzu | 2021-03-24 02:40:34,207 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
g0zhkp446s-algo-1-duwzu | 2021-

RuntimeError: Failed to run: ['docker-compose', '-f', '/private/var/folders/cz/s5n0rss95_d_fkgnjz19kgbr0000gn/T/tmpfdyykckx/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 1

## Train on Cloud Amazon SageMaker

Training

In [31]:
from sagemaker.tensorflow import TensorFlow
from sagemaker.debugger import Rule, rule_configs
from sagemaker import get_execution_role

training_file = "labels_train.csv"
validation_file = "labels_val.csv"
tf_estimator = TensorFlow(entry_point='train_no_s3.py', 
                          role="arn:aws:iam::963659202518:role/service-role/AmazonSageMaker-ExecutionRole-20210306T191865",
                          #                           role=get_execution_role(),
                          train_instance_count=1, 
                          train_instance_type='ml.p3.2xlarge', # ml.m5.large, ml.p3.16xlarge, ml.p3.2xlarge, ml.g4dn.xlarge, ml.p3.8xlarge, p3.2xlarge, ml.t2.micro
                          framework_version='2.4', 
                          base_job_name='pc-tf-custom-container-test-job-RGBNIRN',
                          output_path='s3://canopy-production-ml-output',
                          py_version='py37',
                          checkpoint_s3_uri="s3://canopy-production-ml-output/ckpt/pc-tf-custom-container-test-job-RGBNIRN",
                          script_mode=True,
                          train_use_spot_instances=True,        # Use spot instance
                          train_max_run=432000,                    # Max training time
                          train_max_wait=450000,                  # Max training time + spot waiting time
                          hyperparameters={'training_file':training_file, 
                                           'validation_file': validation_file,
#                                            'bucket':"margaux-bucket-us-east-1",
                                           'epochs': 5, 'augment': True, 'batch-size': 100,
                                           'learning-rate': 0.001, 'numclasses': 10,
                                           'wandb_key': "6607ed7a49b452c2f3494ce60f9514f6c9e3b4e6",
                                           's3_chkpt_dir':"ckpt",
                                           'bands': "2 3 4 8 12",
                                          'starting_checkpoint':None
                                          }
                         )



train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_use_spot_instances has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_wait has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [ ]:
tf_estimator.fit("s3://canopy-production-ml/chips/cloudfree-merge-polygons/split/")


's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2021-03-22 22:33:23 Starting - Starting the training job...
2021-03-22 22:33:24 Starting - Launching requested ML instances.........
2021-03-22 22:35:05 Starting - Preparing the instances for training...
2021-03-22 22:35:53 Downloading - Downloading input data.

# FSX Input Obj

In [14]:
# old_dataset_directory = '/h4zifbmv/chips/cloudfree-merge-polygons/split'
new_dataset_directory = '/h4zifbmv/chips/cloudfree-merge-polygons/dataset_v2'


fsx_data_obj = FileSystemInput(file_system_id='fs-03cd4325554338c21',
                                    file_system_type='FSxLustre',
                                    directory_path=new_dataset_directory,
                                    file_system_access_mode='ro')

# EFS Input Obj

In [38]:
# old_dataset_directory = '/h4zifbmv/chips/cloudfree-merge-polygons/split'
new_dataset_directory = "/"


efs_data_obj = FileSystemInput(file_system_id='fs-f1777e44',
                                    file_system_type='EFS',
                                    directory_path=new_dataset_directory,
                                    file_system_access_mode='ro')

In [39]:
#TODO : LR update - min .0001 vs .00001, Data Aug - Enable "flip_left_right" 



from sagemaker.tensorflow import TensorFlow
from sagemaker.debugger import Rule, rule_configs
from sagemaker import get_execution_role
import time

job_name = 'pc-tf-custom-container-test-job-RGBNIRSWIR1SWIR2'


# old_training_file = "labels_full_train_v6.csv"
# old_validation_file = "labels_val.csv"
# new_training_file = "new_train_labels_v5_10_percent.csv"
# new_training_file = "new_train_labels_v7_one_quarter.csv"
new_training_file = "new_train_labels_v7_one_quarter.csv"
new_validation_file = "new_val_labels_v3.csv"
# old_band_combo = [2,3,4,8,12]
# band_rgbnir = [2,3,4,8,18]
# bands_all = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 16, 17, 18]


tf_estimator = TensorFlow(entry_point='train_no_s3.py', 
                          role="arn:aws:iam::963659202518:role/service-role/AmazonSageMaker-ExecutionRole-20210306T191865",
                          #                           role=get_execution_role(),
                          instance_count=1, 
                          instance_type='ml.g4dn.4xlarge', # ml.m5.large, ml.p3.16xlarge, ml.p3.2xlarge, ml.g4dn.xlarge, ml.p3.8xlarge, p3.2xlarge, ml.t2.micro
                          framework_version='2.4', 
                          base_job_name=job_name,
                          output_path='s3://canopy-production-ml-output',
                          py_version='py37',
                          checkpoint_s3_uri=f's3://canopy-production-ml-output/ckpt/{job_name}-{time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())}',
                          script_mode=True,
                          use_spot_instances=True,        # Use spot instance
                          max_run=432000,                    # Max training time
                          max_wait=450000,                  # Max training time + spot waiting time
                            subnets=["subnet-815befde"],
                              security_group_ids=["sg-0df3ac3b4c291c080"],
                          hyperparameters={'training_file':new_training_file, 
                                           'validation_file': new_validation_file,
#                                            'bucket':"margaux-bucket-us-east-1",
                                           'epochs': 200, 'augment': False, 'batch-size': 64,
                                           'learning-rate': 0.001, 'numclasses': 5,
                                           'wandb_key': "abfa0dec9fc06fbfa6392496f40a22a8d47e58cf",
                                           's3_chkpt_dir':"ckpt",
                                           'bands': "2 3 4 8 11 12 18",
                                          'starting_checkpoint':None, 
                                           'freeze_bn_layer':False
                                          }
                         )

In [ ]:
tf_estimator.fit(efs_data_obj)

2021-04-08 19:20:55 Starting - Starting the training job...
2021-04-08 19:21:19 Starting - Launching requested ML instancesProfilerReport-1617909654: InProgress
.........
2021-04-08 19:22:39 Starting - Preparing the instances for training......
2021-04-08 19:23:59 Downloading - Downloading input data
2021-04-08 19:23:59 Training - Downloading the training image............
2021-04-08 19:26:00 Training - Training image download completed. Training in progress.2021-04-08 19:25:50.520833: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-04-08 19:25:50.525507: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-04-08 19:25:50.616182: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-08 19:25:50.702849: W tensorflow/core/p

# Sandbox

In [62]:
from tensorflow_addons.metrics import F1Score, HammingLoss
from tensorflow_addons.losses import SigmoidFocalCrossEntropy


# model_path = "/Users/zwarshavsky/Downloads/model-best.h5"

model_path = "model_weights_RGBNIRN.h5"

dependencies = {
    'F1Score': F1Score
}

# model = keras.models.load_model(self.output_directory + 'best_model.hdf5', custom_objects=dependencies)

model = tf.keras.models.load_model(model_path, custom_objects=dependencies)

ValueError: No model found in config file.

In [61]:
model.layers

# Temp

In [69]:
# label_list = ["Habitation", "Industrial_agriculture", "Mining", "Rainforest", "River", "Roads", "Savannah", 
#               "Shifting_cultivation", "Water"]

new_label_list = ["Industrial_agriculture","ISL","Mining","Roads","Shifting_cultivation"]

# 1:"Industrial_agriculture"
# 2:"ISL"
# 3:"Mining"
# 4:"Roads"
# 5:"Shifting_cultivation"


numclasses = len(new_label_list)
bands_all = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 16, 17, 18]
bands_RGBNIRN = [2, 3, 4, 8, 12]
bands_RGBNIRSN = [2, 3, 4, 8, 12, 18]
# input_shape_all_bands= (100,100,len(bands_all))
input_shape_RGBNIRN = (100,100,len(bands_RGBNIRN))
input_shape_all = (100,100,len(bands_all))

In [ ]:
import wget

s3 = boto3.resource('s3')
model_url = "https://api.wandb.ai/files/zwarshavsky/project-canopy/pc-tf-custom-container-test-job-RGBNIRN-2021-04-06-20-28-35-428-algo-1/model-best.h5"
checkpoint_url = "s3://canopy-production-ml-output/ckpt/pc-tf-custom-container-test-job-RGBNIRN-2021-04-06-20-28-35-428/model_resnet_epoch_19.h5"
model_weights_filename = "model_weights.h5"

In [ ]:
#Download Model 
model_path = wget.download(model_url)

#Download Weights
KEY = "/".join(checkpoint_url.split("/")[3:])
s3.Bucket('canopy-production-ml-output').download_file(KEY, model_weights_filename)

In [ ]:
model_test = tf.keras.models.load_model(model_path)
model_test.load_weights(model_weights_filename)

In [ ]:
batch_size = 128
# csv_path = "labels_test.csv" 
# new_csv_path = "new_test_labels_v2.csv" 
# new_csv_path_one_third = "new_test_labels_v3_one_third.csv"
train_set = "new_train_labels_v7_one_quarter.csv"
test_set = "new_test_labels_v3_one_third_uri_only.csv"
val_labels = "new_val_labels_v3.csv"

# gen_all_bands = TestGenerator(label_file_path_test=test_set,
#                               training_dir = "./efs/",
#                          bucket_name='canopy-production-ml',
#                          label_mapping_path="new_labels.json",
#                          data_extension_type='.tif',
#                               file_mode="file",
#                          bands=bands_all,
#                          test_data_shape=input_shape_all,
#                          test_data_batch_size=batch_size,
#                          enable_data_prefetch=False,
#                          data_prefetch_size=tf.data.experimental.AUTOTUNE,
#                          num_parallel_calls=2 * multiprocessing.cpu_count(),
#                          output_shape=(tf.float32, tf.float32))

gen_RGBNIRN = TestGenerator(label_file_path_test=test_set,
                            training_dir = "./efs",
                         bucket_name='canopy-production-ml',
                         label_mapping_path="new_labels.json",
                         data_extension_type='.tif',
                            file_mode="file",
                         bands=bands_RGBNIRN,
                         test_data_shape=input_shape_RGBNIRN,
                         test_data_batch_size=batch_size,
                         enable_data_prefetch=False,
                         data_prefetch_size=tf.data.experimental.AUTOTUNE,
                         num_parallel_calls=2 * multiprocessing.cpu_count(),
                         output_shape=(tf.float32, tf.float32))

In [ ]:
model_test.evaluate(gen_RGBNIRN.validation_dataset)